In [1]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from os import listdir
import pandas as pd

In [15]:
# Reding the name of all archives in the results directory
arc_list = listdir('results/')
results_list = []

# Separating results types from the archive names
for arc in arc_list:
    result_type = arc.split('__')[0]
    
    if result_type != 'target' and result_type != 'bayes_from_attributes' and result_type != 'posterior_probability':
        results_list.append(result_type)
    
# Filtering the unique names:
results_list = np.unique(results_list)

print('The all the results comprised by this analysis are: {}'.format(results_list))

# Creating a dictionary of the lists
# of archives for each result type

results_dict = {}

for res in results_list:
    results_dict[res] = []
    
# Appending the files names for each result 
# and countting number of granularities

n_gra = 0
n_it = 0

for arc in arc_list:
    result_type = arc.split('__')[0]
    
    if result_type != 'target' and result_type != 'bayes_from_attributes' and result_type != 'posterior_probability':
        results_dict[result_type].append(arc)
        
        iteration = int(arc.split('__')[2])
        granularity = int(arc.split('__')[1])

        if granularity > n_gra:
            n_gra = granularity
            
        if iteration > n_it:
            n_it = iteration

# Iterations begin in zero, therefore:

n_it += 1

The all the results comprised by this analysis are: ['Density_1' 'Density_2' 'SODA_labels' 'performance_info']


In [145]:
y = np.genfromtxt('results/target__.csv')
density_df = {
    'D1': {
        'mean': np.zeros((len(y))),
        'std': 0
    },
    'D2': {
        'mean': np.zeros((len(y))),
        'std': 0
    }
}


for arc in results_dict['Density_1']:
    gra = arc.split('__')[1]
    it = int(arc.split('__')[2])
    if gra == '4':
        D = np.genfromtxt('results/'+arc,delimiter=',')
        density_df['D1']['mean'][:] = D
        
for arc in results_dict['Density_2']:
    gra = arc.split('__')[1]
    it = int(arc.split('__')[2])
    if gra == '4':
        D = np.genfromtxt('results/'+arc,delimiter=',')
        density_df['D2']['mean'][:] = D
        
density_df['D1']['std']= np.std(density_df['D1']['mean'],axis = 0)
#density_df['D1']['mean']= np.mean(density_df['D1']['mean'],axis = 0)
density_df['D2']['std']= np.std(density_df['D2']['mean'],axis = 0)
#density_df['D2']['mean']= np.mean(density_df['D2']['mean'],axis = 0)
print(density_df['D1']['mean'].shape)

(100000,)


In [148]:
results_dict['SODA_labels']

['SODA_labels__4__5__.csv',
 'SODA_labels__4__9__.csv',
 'SODA_labels__8__1__.csv',
 'SODA_labels__3__17__.csv',
 'SODA_labels__1__28__.csv',
 'SODA_labels__2__31__.csv',
 'SODA_labels__4__8__.csv',
 'SODA_labels__3__24__.csv',
 'SODA_labels__2__6__.csv',
 'SODA_labels__2__12__.csv',
 'SODA_labels__1__5__.csv',
 'SODA_labels__3__20__.csv',
 'SODA_labels__2__1__.csv',
 'SODA_labels__3__25__.csv',
 'SODA_labels__8__6__.csv',
 'SODA_labels__6__6__.csv',
 'SODA_labels__6__28__.csv',
 'SODA_labels__6__8__.csv',
 'SODA_labels__4__11__.csv',
 'SODA_labels__2__0__.csv',
 'SODA_labels__5__28__.csv',
 'SODA_labels__6__9__.csv',
 'SODA_labels__4__22__.csv',
 'SODA_labels__7__4__.csv',
 'SODA_labels__1__15__.csv',
 'SODA_labels__6__26__.csv',
 'SODA_labels__3__3__.csv',
 'SODA_labels__2__18__.csv',
 'SODA_labels__8__13__.csv',
 'SODA_labels__8__30__.csv',
 'SODA_labels__5__30__.csv',
 'SODA_labels__2__8__.csv',
 'SODA_labels__4__3__.csv',
 'SODA_labels__1__32__.csv',
 'SODA_labels__3__14__.csv',
 

In [149]:
# Gathering number of DC for each granularities
arc = results_dict['SODA_labels'][2]

label = np.genfromtxt('results/'+arc,delimiter=',')
a_dc = np.unique(label[y==1])
n_dc_index = []
for i in range(len(label)):
        if label[i] not in a_dc: 
            n_dc_index.append(i)

n_dc, n_normal_dc = np.unique(label[n_dc_index],return_counts=True)

anomaly_dc_df = pd.DataFrame(np.zeros((len(a_dc),5)),columns = ['ID','n_samples','percent_anomaly',
                                                                      'percent_background','percent_training'])

normal_dc_df = pd.DataFrame(np.zeros((len(n_dc),5)),columns = ['ID','n_samples','percent_anomaly',
                                                                      'percent_background','percent_training'])
anomaly_dc_df['ID'] = a_dc
normal_dc_df['ID'] = n_dc

normal_dc_df['n_samples'] = n_normal_dc

for i in range(len(label)):
    for j in range(len(anomaly_dc_df['ID'])):
        if label[i] == anomaly_dc_df['ID'][j]:
            anomaly_dc_df['n_samples'][j] += 1
            
            if y[i] == -1:
                anomaly_dc_df['percent_training'][j] += 1
            elif y[i] == 0:
                anomaly_dc_df['percent_background'][j] += 1
            elif y[i] == 1:
                anomaly_dc_df['percent_anomaly'][j] += 1
                
for i in range(len(label)):
    for j in range(len(normal_dc_df['ID'])):
        if label[i] == normal_dc_df['ID'][j]:            
            if y[i] == -1:
                normal_dc_df['percent_training'][j] += 1
            elif y[i] == 0:
                normal_dc_df['percent_background'][j] += 1
            
anomaly_dc_df['percent_anomaly'] /= anomaly_dc_df['n_samples']
anomaly_dc_df['percent_background'] /= anomaly_dc_df['n_samples']
anomaly_dc_df['percent_training'] /= anomaly_dc_df['n_samples']

normal_dc_df['percent_anomaly'] /= normal_dc_df['n_samples']
normal_dc_df['percent_background'] /= normal_dc_df['n_samples']
normal_dc_df['percent_training'] /= normal_dc_df['n_samples']

anomaly_dc_df['percent_anomaly'] *= 100
anomaly_dc_df['percent_background'] *= 100
anomaly_dc_df['percent_training'] *= 100

normal_dc_df['percent_anomaly'] *= 100
normal_dc_df['percent_background'] *= 100
normal_dc_df['percent_training'] *= 100

/home/thiago/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/thiago/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [158]:
anomaly_dc_df

,ID,n_samples,percent_anomaly,percent_background,percent_training
0,0.0,2328.0,0.730241,31.314433,67.955326
1,1.0,1743.0,0.803213,32.702238,66.494550
2,2.0,1566.0,0.255428,30.906769,68.837803
3,3.0,2010.0,0.547264,25.273632,74.179104
4,4.0,1706.0,0.703400,29.484174,69.812427
5,5.0,1311.0,0.152555,32.265446,67.581998
6,6.0,1358.0,0.294551,32.474227,67.231222
7,7.0,448.0,0.892857,34.598214,64.508929
8,8.0,1496.0,0.401070,30.080214,69.518717
9,9.0,1224.0,0.816993,33.415033,65.767974


In [157]:
pd.set_option("display.max_rows", None)
